In [6]:
# Import all the libs
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import tensorflow_hub as hub
import tensorflow_datasets as tfds

from tensorflow.keras import layers

import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [12]:
IMAGE_RES = 299 # Since input of Incep V3 is 299 x 299 

(train_set, valid_set), dataset_info = tfds.load(
    'tf_flowers',
    split=['train[:70%]', 'train[70%:]'],
    with_info=True,
    as_supervised=True,
)

# Get number of classes and number of training examples / valid examples (labels)
num_classes = dataset_info.features['label'].num_classes

num_train_examples = 0
num_valid_examples = 0

for example in train_set:
  num_train_examples += 1

for example in valid_set:
  num_valid_examples += 1

In [15]:

# Functions to reformat the image
def format_image(image, label):
    image = tf.image.resize(image, (IMAGE_RES, IMAGE_RES))/255.0
    return image, label

In [16]:

BATCH_SIZE = 32
train_batches = train_set.shuffle(num_train_examples).map(format_image).batch(BATCH_SIZE).prefetch(1)
valid_batches = valid_set.map(format_image).batch(BATCH_SIZE).prefetch(1)

In [17]:
# Feature extractor
URL = "https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4"
feature_extractor = hub.KerasLayer(URL,
  input_shape=(IMAGE_RES, IMAGE_RES, 3),
  trainable=False)

In [18]:
# Create compile and fit the model
model_inception = tf.keras.Sequential([
  feature_extractor,
  tf.keras.layers.Dense(num_classes)
])

model_inception.summary()

model_inception.compile(
  optimizer='adam', 
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

EPOCHS = 6

history = model_inception.fit(train_batches,
                    epochs=EPOCHS,
                    validation_data=valid_batches)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 2048)              21802784  
                                                                 
 dense (Dense)               (None, 5)                 10245     
                                                                 
Total params: 21813029 (83.21 MB)
Trainable params: 10245 (40.02 KB)
Non-trainable params: 21802784 (83.17 MB)
_________________________________________________________________
Epoch 1/6
81/81 [==============================] - 125s 2s/step - loss: 0.7322 - accuracy: 0.7396 - val_loss: 0.4543 - val_accuracy: 0.8629
Epoch 2/6
81/81 [==============================] - 129s 2s/step - loss: 0.3884 - accuracy: 0.8735 - val_loss: 0.3570 - val_accuracy: 0.8837
Epoch 3/6
81/81 [==============================] - 130s 2s/step - loss: 0.3055 - accuracy: 0.9035 - val_loss: 0.3340 - val_accu